<a href="https://colab.research.google.com/github/agorastats/mitosisDEEP/blob/master/colabs/dataGeneratorTrain_2022_03_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages
import os
import tensorflow as tf
import numpy as np 
import pandas as pd 
import random
import cv2
from tensorflow import keras

In [11]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [3]:
# get data from google drive (unzip images to read it)
# !unzip 'drive/MyDrive/mitosis_data/dataset_20220207.zip' -d "/content/data"/
!unzip 'drive/MyDrive/mitosis_data/all_data_2022_03_20.zip' -d "/content/data"/

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/data/masks/078_49.jpg  
  inflating: /content/data/masks/078_55.jpg  
  inflating: /content/data/masks/078_56.jpg  
  inflating: /content/data/masks/079_1.jpg  
  inflating: /content/data/masks/080_0.jpg  
  inflating: /content/data/masks/081_0.jpg  
  inflating: /content/data/masks/083_1.jpg  
  inflating: /content/data/masks/084_1.jpg  
  inflating: /content/data/masks/084_4.jpg  
  inflating: /content/data/masks/084_16.jpg  
  inflating: /content/data/masks/084_23.jpg  
  inflating: /content/data/masks/085_1.jpg  
  inflating: /content/data/masks/085_3.jpg  
  inflating: /content/data/masks/086_4.jpg  
  inflating: /content/data/masks/086_8.jpg  
  inflating: /content/data/masks/086_14.jpg  
  inflating: /content/data/masks/086_18.jpg  
  inflating: /content/data/masks/090_4.jpg  
  inflating: /content/data/masks/091_1.jpg  
  inflating: /content/data/masks/092_5.jpg  
  inflating: /content/data/masks

In [4]:
# import mitosisDEEP github project
!rm -rf mitosisDEEP  # remove if exists
# Clone the entire repo
!git clone -l -s https://github.com/agorastats/mitosisDEEP.git mitosisDEEP
import sys
sys.path.insert(0, 'mitosisDEEP/')

Cloning into 'mitosisDEEP'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 500 (delta 240), reused 360 (delta 114), pack-reused 0
Receiving objects: 100% (500/500), 113.20 MiB | 30.58 MiB/s, done.
Resolving deltas: 100% (240/240), done.


In [5]:
# import generator and needed packages
!pip install opencv-python.headless~=4.1.2.30
!pip install albumentations~=1.1.0

from mitosisDEEP.dataGeneratorProcess import DataGenerator

df=pd.read_csv('data/infoDF.csv', sep=';')
gen = DataGenerator(df=df, img_path='data/images', mask_path='data/masks', shuffle=False)

     |████████████████████████████████| 21.8 MB 54.3 MB/s 
     |████████████████████████████████| 102 kB 6.8 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1828: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1802: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,


In [6]:
# keras utils
from mitosisDEEP.utils.keras.lossAndMetrics import dice_coef, bce_dice_loss

In [7]:
# PRETRAINED UNET MODEL
from keras.models import load_model
model = load_model("drive/MyDrive/mitosis_data/UNET-Best.h5", compile=False)
# model.save_weights("drive/MyDrive/mitosis_data/pretrained_weights.h5")

opt_adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt_adam, loss=bce_dice_loss, metrics=[dice_coef])
# model.load_weights("drive/MyDrive/mitosis_data/pretrained_weights.h5")
model.load_weights("drive/MyDrive/mitosis_data/all_data_2022_03_20_unet.h5")
# model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet.h5")
# model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_pretrained_unet_v2.h5")
# model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_unet_bce_dice_loss.h5")

In [ ]:
# https://www.kaggle.com/basu369victor/transferlearning-and-unet-to-segment-rocks-on-moon
# FREEZE LAYERS
for i in [4, 9, 12, 17, 20, 25, 28, 33, 36]:  
  model.layers[i].trainable = False

layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
infoModelDF = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
# PRETRAINED BACKBONE MODEL
!pip install segmentation-models
import segmentation_models as sm
# per evitar error set framework keras: https://stackoverflow.com/questions/67792138/attributeerror-module-keras-utils-has-no-attribute-get-file-using-segmentat
sm.set_framework('tf.keras')
sm.framework()
# LOAD UNET WITH RESNET34 (IMAGENET)
BACKBONE = 'resnet34'
model = sm.Unet(BACKBONE, input_shape=(256, 256, 3), classes=1, activation='sigmoid')  #  encoder_freeze=True 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=bce_dice_loss, metrics=[dice_coef])

preprocess_input = sm.get_preprocessing(BACKBONE) # resnet34 not make special transformation, it uses /255.


# proves_dataset20220207_resnet_bce_dice_loss

# model.load_weights("drive/MyDrive/mitosis_data/init_backbone_bce_dice_loss.h5")
# model.load_weights("drive/MyDrive/mitosis_data/backbone_resnet.h5")
# model.load_weights("drive/MyDrive/mitosis_data/backbone_resnet_dice_loss.h5")
# model.load_weights("drive/MyDrive/mitosis_data/proves_dataset20220207_backbone_net.h5")

In [12]:
from mitosisDEEP.utils.keras.callbacks import get_callbacks_list

output_name = 'all_data_2022_03_20_unet'
callbacks = get_callbacks_list(output_name, N=5)

In [13]:
# random choice for val images
idx = np.random.choice(len(df), int(len(df) * 0.9), replace=False)
trainDF = df.iloc[idx, :]
valDF = df.iloc[~idx, :]
# trainDF = df.loc[df.loc[:, 'images_from'].isin(['data/icpr12', 'data/BreCaHAD']), :]
# valDF = df.loc[~df.loc[:, 'images_from'].isin(['data/icpr12', 'data/BreCaHAD']), :]  # AMIDA13 to validate

training_generator = DataGenerator(df=trainDF, batch_size=64, img_path='data/images', mask_path='data/masks', 
                                   shuffle=True, preprocess=None) 
validation_generator = DataGenerator(df=valDF, batch_size=64,  img_path='data/images', mask_path='data/masks',
                                     preprocess=None, augmentations=None, shuffle=False)    # preprocess_input
                                     
EPOCH_NUMBER=100


history = model.fit(training_generator, validation_data=validation_generator, verbose=1,
                    epochs=EPOCH_NUMBER, callbacks=callbacks) 


Epoch 1/100
98/98 [==============================] - ETA: 0s - loss: 0.2352 - dice_coef: 0.7838
Epoch 1: val_loss improved from inf to 0.20119, saving model to /content/drive/MyDrive/mitosis_data/all_data_2022_03_20_unet.h5
98/98 [==============================] - 108s 983ms/step - loss: 0.2352 - dice_coef: 0.7838 - val_loss: 0.2012 - val_dice_coef: 0.8144 - lr: 1.0000e-04
Epoch 2/100
98/98 [==============================] - ETA: 0s - loss: 0.2343 - dice_coef: 0.7844
Epoch 2: val_loss improved from 0.20119 to 0.19960, saving model to /content/drive/MyDrive/mitosis_data/all_data_2022_03_20_unet.h5
98/98 [==============================] - 97s 986ms/step - loss: 0.2343 - dice_coef: 0.7844 - val_loss: 0.1996 - val_dice_coef: 0.8159 - lr: 1.0000e-04
Epoch 3/100
98/98 [==============================] - ETA: 0s - loss: 0.2386 - dice_coef: 0.7806
Epoch 3: val_loss did not improve from 0.19960
98/98 [==============================] - 96s 981ms/step - loss: 0.2386 - dice_coef: 0.7806 - val_loss:

In [13]:
# evaluate mitosis
# ref: https://www.youtube.com/watch?v=HrGn4uFrMOM
from google.colab.patches import cv2_imshow
!pip install patchify
from patchify import patchify, unpatchify
PATCH_SIZE  = 256
# test_path = 'drive/MyDrive/mitosis_data/A00_01.bmp'
# test_path = 'drive/MyDrive/mitosis_data/A01_09.bmp'  # validation
test_path = '/content/drive/MyDrive/mitosis_data/UOC_Mitosis/24.jpg'   
# test_path = 'drive/MyDrive/mitosis_data/23.jpg'   
# test_path = 'drive/MyDrive/mitosis_data/25.jpg'   
from mitosisDEEP.utils.image import read_image
test_image = read_image(test_path)
# cv2_imshow(test_image)

In [8]:
# predict using patchify
size_x = (test_image.shape[1]//PATCH_SIZE)*PATCH_SIZE   # nearest size divisible by our patch size
size_y = (test_image.shape[0]//PATCH_SIZE)*PATCH_SIZE   # nearest size divisible by our patch size
print('test image size: ', test_image.shape)
test_image = cv2.resize(test_image, (size_x, size_y))
print('test image resized size: ', test_image.shape)

patches = patchify(test_image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_SIZE)  # step same as patch for not overlap patches
patches = patches[:, :, 0, :, :, :]
predicted_patches = []
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        single_patch = patches[i,j,:,:, :] / 255.   # normalize
        single_patch = np.expand_dims(single_patch, axis=0) # (256,256,3) to (1,256,256,3)
        single_patch_prediction = (model.predict(single_patch) > 0.5).astype(np.uint8)
        predicted_patches.append(single_patch_prediction[0, :, :])


predicted_patches_reshaped = np.reshape(predicted_patches, [patches.shape[0], patches.shape[1], patches.shape[2], patches.shape[3]]) 
reconstructed_image = unpatchify(predicted_patches_reshaped, (test_image.shape[0], test_image.shape[1]))
# cv2_imshow(reconstructed_image * 255.) # multiply to visualize graph


test image size:  (1173, 2560, 3)
test image resized size:  (1024, 2560, 3)


In [12]:
# MITOSIS UOC
# path_test = '/content/drive/MyDrive/mitosis_data/UOC_Mitosis'
# testDF= pd.DataFrame({'id': os.listdir(path_test)})
# testDF.head(3)

# LAURA MITOSIS
# path_test = '/content/drive/MyDrive/mitosis_data/LauraMitosis'
# testDF= pd.DataFrame({'id': os.listdir(path_test)})
# testDF.head(3)

# ICPR12 EVALUATION 
path_test = '/content/drive/MyDrive/mitosis_data/icpr12_evaluation'
testDF= pd.DataFrame({'id': os.listdir(path_test)})
testDF.head(3)


,id
0,H04_03.bmp
1,H01_06.bmp
2,A04_09.bmp


In [22]:
!pip install patchify
from mitosisDEEP.evaluateLargeImageProcess import EvaluateLargeImageProcess

# with STAIN
evalImg = EvaluateLargeImageProcess(df=testDF, img_path=path_test, model=model, stain=True, output_info='output', 
stain_ref_img='mitosisDEEP/utils/stain/A00_01_ref_img.bmp')

# without STAIN

In [23]:
import logging
logging.getLogger().setLevel(logging.INFO)
evalImg.run(dict())

INFO:root:predict image: 036.jpg
INFO:root:__predict with stain: False
INFO:root:___test image size: (1920, 1017)
INFO:root:___test image resized size: (1792, 768)
INFO:root:____predict patches
INFO:root:____reconstruct image with patches
INFO:root:__predict with stain: True
INFO:root:___test image size: (1920, 1017)
INFO:root:___test image resized size: (1792, 768)
INFO:root:____predict patches
INFO:root:____reconstruct image with patches
INFO:root:predict image: 023.jpg
INFO:root:__predict with stain: False
INFO:root:___test image size: (1920, 1017)
INFO:root:___test image resized size: (1792, 768)
INFO:root:____predict patches
INFO:root:____reconstruct image with patches
INFO:root:__predict with stain: True
INFO:root:___test image size: (1920, 1017)
INFO:root:___test image resized size: (1792, 768)
INFO:root:____predict patches
INFO:root:____reconstruct image with patches
INFO:root:predict image: 025.jpg
INFO:root:__predict with stain: False
INFO:root:___test image size: (1920, 1017